In [1]:
import pandas as pd
import numpy as np
import os

## Define file path depending on which index to examine

## SOX
index_file_path = "H:/Tech Hardware Shared/$Mike/Quant/SOX_Constit.csv"
csv_directory = "H:/Tech Hardware Shared/$Mike/Quant/CSV_files/Updated"
export_path = "H:/Tech Hardware Shared/$Mike/Quant/Python_Outputs/Updated/"
export_audit_file_name = 'SOX_audit_updated.xlsx'
export_index_file_name = 'SOX_Index_Avg_Updateds.xlsx'
export_LS_file_name = 'SOX_LS_Updated.xlsx'
export_dispersion_file_name = 'SOX_dispersion_Updated.xlsx'
export_examine_CSV_file_name = 'SOX_indiv_CSV_Updated.xlsx'

##### CREATING DATABASE DATAFRAME ######

# Read the CSV file into a DataFrame
df = pd.read_csv(index_file_path, header=0)
# Convert the column names to datetime with the given format
df.columns = pd.to_datetime(df.columns, format='%d-%b-%y')

# Melt the DataFrame to reshape it
df = pd.melt(df, id_vars=[], var_name='Date', value_name='ISIN')
df['Date'] = df['Date'] + pd.offsets.MonthEnd(0)

# Use a placeholder for NaN values in the 'ISIN' column 
df['ISIN'].fillna('placeholder', inplace=True)

# Create a list to store each ISIN's data
all_isin_data = []

# Iterate through each ISIN in the 'ISIN' column 
for isin in df['ISIN'].unique():
    # Skip if the ISIN is 'placeholder'
    if isin != 'placeholder':
        # Construct the file path for the CSV file
        index_file_path = os.path.join(csv_directory, f'{isin}.csv')
        # Check if the CSV file exists
        if os.path.exists(index_file_path):        
            # Read the CSV file
            isin_data = pd.read_csv(index_file_path, header=0)  # Assuming header is in row 2 and 'Date' is the label
            isin_data['Date'] = pd.to_datetime(isin_data['Date'], unit='D', origin='1899-12-30')
            isin_data['Date'] = isin_data['Date'] + pd.offsets.MonthEnd(0)
            isin_data['ISIN'] = isin
            
            # Convert all data columns to numeric                   
            numeric_columns = ['Mkt_Cap','EV_NTM','RI','P','NOSH','Reccon',
                               'Rev_LTM','Rev_NTM','Rev_2yFwd',
                               'EBITDA_LTM','EBITDA_NTM','EBITDA_2yFwd',
                               'EPS_LTM','EPS_NTM','EPS_2yFwd',
                               'BPS_LTM','BPS_NTM','BPS_2yFwd',
                               'DPS_LTM','DPS_NTM','DPS_2yFwd',
                               'CFPS_LTM','CF_LTM_DS','CFPS_NTM',
                               'Assets_LTM',
                               'NetDebt_LTM','NetDebt_LTM_DS','NetDebt_NTM']            
            isin_data[numeric_columns] = isin_data[numeric_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
            
            
            
            # Data cleanse
            isin_data['NetDebt_LTM_Best'] = np.where(pd.notnull(isin_data['NetDebt_LTM']),isin_data['NetDebt_LTM'],isin_data['NetDebt_LTM_DS'])
            # nix out negative BVPS data
            #neg_cols = ['BY_TTM','PB_TTM', 'BY_1y_Fwd','PB_1y_Fwd','BY_2y_Fwd','PB_2y_Fwd','CoE']
            #for col in neg_cols:
            #    isin_data[col] = np.where(isin_data[col] < 0, np.nan, isin_data[col]) 
            
            isin_data['Mkt_Cap_SOM'] = isin_data['Mkt_Cap'].shift(1)
            isin_data['TR'] = isin_data['RI'] / isin_data['RI'].shift(1) -1
            isin_data['TR_LTM'] = isin_data['RI'] / isin_data['RI'].shift(12) -1
            isin_data['TR_L6M'] = isin_data['RI'] / isin_data['RI'].shift(6) -1
            isin_data['TR_L3M'] = isin_data['RI'] / isin_data['RI'].shift(3) -1
            isin_data['PCH'] = isin_data['P'] / isin_data['P'].shift(1) -1
            isin_data['PCH_LTM'] = isin_data['P'] / isin_data['P'].shift(12) -1
            isin_data['PCH_L6M'] = isin_data['P'] / isin_data['P'].shift(6) -1           
            isin_data['PCH_L3M'] = isin_data['P'] / isin_data['P'].shift(3) -1           
            isin_data['NTM_RevGrowth'] = isin_data['Rev_NTM'] / isin_data['Rev_LTM'] - 1
            isin_data['2y_RevCAGR'] = (isin_data['Rev_2yFwd'] / isin_data['Rev_LTM'])**(1/2) - 1
            isin_data['NTM_RevGrowth_3mChg'] = isin_data['NTM_RevGrowth'] - isin_data['NTM_RevGrowth'].shift(3)
            isin_data['NTM_Rev_3mChg'] = isin_data['Rev_NTM'] / isin_data['Rev_NTM'].shift(3) - 1
            isin_data['LTM_EBITDA_Margin'] = isin_data['EBITDA_LTM'] / isin_data['Rev_LTM']
            isin_data['NTM_EBITDA_Margin'] = isin_data['EBITDA_NTM'] / isin_data['Rev_NTM']
            isin_data['2yFwd_EBITDA_Margin'] = isin_data['EBITDA_2yFwd'] / isin_data['Rev_2yFwd']
            isin_data['NTM_EBITDA_Margin_3mChg'] = isin_data['NTM_EBITDA_Margin'] - isin_data['NTM_EBITDA_Margin'].shift(3)
            isin_data['RoE_LTM'] = isin_data['EPS_LTM'] / isin_data['BPS_LTM']
            isin_data['RoE_NTM'] = isin_data['EPS_NTM'] / isin_data['BPS_NTM']
            isin_data['RoE_2yFwd'] = isin_data['EPS_2yFwd'] / isin_data['BPS_2yFwd']
            isin_data['Sales_EV_NTM'] = isin_data['Rev_NTM'] / isin_data['EV_NTM']
            isin_data['EBITDA_EV_NTM'] = isin_data['EBITDA_NTM'] / isin_data['EV_NTM']
            isin_data['EV_LTM'] = isin_data['Mkt_Cap'] + isin_data['NetDebt_LTM_Best']
            isin_data['EY_LTM'] = isin_data['EPS_LTM'] / isin_data['P']
            isin_data['EY_NTM'] = isin_data['EPS_NTM'] / isin_data['P']
            isin_data['EY_2yFwd'] = isin_data['EPS_2yFwd'] / isin_data['P']
            isin_data['BY_LTM'] = isin_data['BPS_LTM'] / isin_data['P']
            isin_data['BY_NTM'] = isin_data['BPS_NTM'] / isin_data['P']
            isin_data['BY_2yFwd'] = isin_data['BPS_2yFwd'] / isin_data['P']
            isin_data['DY_LTM'] = isin_data['DPS_LTM'] / isin_data['P']
            isin_data['DY_NTM'] = isin_data['DPS_NTM'] / isin_data['P']
            isin_data['DY_2yFwd'] = isin_data['DPS_2yFwd'] / isin_data['P']
            isin_data['NOSH_Chg_LTM'] = isin_data['NOSH'] / isin_data['NOSH'].shift(12) - 1
            isin_data['CFY_LTM'] = isin_data['CFPS_LTM'] / isin_data['P']
            isin_data['CFY_LTM_DS'] = isin_data['CF_LTM_DS'] / isin_data['Mkt_Cap']
            isin_data['CFY_NTM'] = isin_data['CFPS_NTM'] / isin_data['P']
            isin_data['CF_Assets'] = isin_data['CF_LTM_DS'] / isin_data['Assets_LTM']
            isin_data['EBITDA_Assets'] = isin_data['EBITDA_LTM'] / isin_data['Assets_LTM']
                                  
            # CoE calculation
                                 
            # Define g
            conditions = [
                (isin_data['RoE_2yFwd'] < 0.1),
                (isin_data['RoE_2yFwd'] >= 0.1) & (isin_data['RoE_2yFwd'] < 0.15),
                (isin_data['RoE_2yFwd'] > 0.15)
            ]
            values = [0, 0.03, 0.05]
            isin_data['Term_G'] = np.select(conditions, values)
            
            # Default CoE to use when no prior CoE is available
            default_CoE = 0.085

            # Initialize the CoE column with NaN values
            isin_data['CoE'] = np.nan

            # Iteratively calculate CoE
            for i in range(len(isin_data)):
                # Get or set CoE for current iteration
                if i == 0 or pd.isnull(isin_data.loc[i - 1, 'CoE']):
                    current_CoE = default_CoE  # Use default CoE for the first iteration or if prior CoE is missing
                else:
                    current_CoE = isin_data.loc[i - 1, 'CoE']

                # Calculate temporary values needed for CoE
                isin_data.loc[i,'Adj_RoE'] = isin_data.loc[i,'RoE_2yFwd'] / (1 + current_CoE)**2
                isin_data.loc[i,'Adj_Price'] = (
                    (1 - (
                        (isin_data.loc[i,'DY_NTM'] + isin_data.loc[i,'NOSH_Chg_LTM']) / (1 + current_CoE) +
                        (isin_data.loc[i,'DY_2yFwd'] + isin_data.loc[i,'NOSH_Chg_LTM']) / (1 + current_CoE)**2
                    )) * isin_data.loc[i,'P']
                )
                isin_data.loc[i, 'Adj_PB'] = isin_data.loc[i, 'Adj_Price'] / isin_data.loc[i,'BPS_NTM']

                # Calculate CoE for the current row
                isin_data.loc[i,'CoE'] = (
                    (isin_data.loc[i,'Adj_RoE'] - isin_data.loc[i,'Term_G']) / isin_data.loc[i,'Adj_PB']
                ) + isin_data.loc[i,'Term_G']
                
            # Implied g calculation
            isin_data['Adj_RoE'] = isin_data['RoE_2yFwd'] / (1 + default_CoE)**2
            isin_data['Adj_Price'] = (1-((isin_data['DY_NTM'] + isin_data['NOSH_Chg_LTM']) / (1 + default_CoE) + 
                                        (isin_data['DY_2yFwd'] + isin_data['NOSH_Chg_LTM']) / (1 + default_CoE)**2))*isin_data['P']
            isin_data['Adj_PB'] = isin_data['Adj_Price'] / isin_data['BPS_NTM']
            
            isin_data['Implied_G'] = (isin_data['Adj_PB']*default_CoE - isin_data['Adj_RoE']) / (isin_data['Adj_PB'] - 1)
            
            # Drop temporary columns after calculation
            isin_data.drop(columns=['Adj_RoE', 'Adj_Price', 'Adj_PB'], inplace=True)

            # Set boundary conditions
            upper_bound = 0.5 # Max CoE = 50%
            isin_data['CoE'] = isin_data['CoE'].clip(upper=upper_bound)
                                   
            lower_bound = -0.02 # Max multiple = -50x
            upper_bound = 1 # Min multiple = 1x
            isin_data['Sales_EV_NTM'] = isin_data['Sales_EV_NTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['EBITDA_EV_NTM'] = isin_data['EBITDA_EV_NTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['EY_LTM'] = isin_data['EY_LTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['EY_NTM'] = isin_data['EY_NTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['EY_2yFwd'] = isin_data['EY_2yFwd'].clip(upper=upper_bound,lower=lower_bound)
            
            lower_bound = -0.5
            upper_bound = 1.5
            isin_data['RoE_LTM'] = isin_data['RoE_LTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['RoE_NTM'] = isin_data['RoE_NTM'].clip(upper=upper_bound,lower=lower_bound)
            isin_data['RoE_2yFwd'] = isin_data['RoE_2yFwd'].clip(upper=upper_bound,lower=lower_bound)
            
            # Revenue & EPS momentum
                        
            # Main function to calculate momentum metrics
            def calculate_momentum(isin_data, base_var, initial_calc_func=None):
                # Dictionary to store new column names for tracking
                new_columns = {}
                # Create a temporary DataFrame for calculations
                temp_df = pd.DataFrame(index=isin_data.index)
                # Perform initial calculations if an initial_calc_func is provided
                if initial_calc_func:
                    for i in range(1, 4):
                        initial_calc_func(isin_data, temp_df, base_var, i)
                # Define numerator and denominator columns based on base_var
                numerator = f'{base_var}_NTM'
                denominators = [f'{base_var}_1.{i}' for i in range(1, 4)]
                # Perform the calculation for each denominator in the temp DataFrame
                for column_name in denominators:
                    result_column = f'F_{column_name}'
                    temp_df[result_column] = np.where(
                        temp_df[column_name] < 0,
                        np.nan,
                        isin_data[numerator] / temp_df[column_name]
                    )
                    new_columns[result_column] = temp_df[result_column]
                # Calculate the momentum metric in temp_df
                momentum_column = f'{base_var}_MTUM_LTM'
                temp_df[momentum_column] = sum(
                    temp_df[f'F_{base_var}_1.{i}'] ** (12 / i) - 1 for i in range(1, 4)
                ) / 3
                # Add only the final momentum column to isin_data
                isin_data[momentum_column] = temp_df[momentum_column]

            # Define the initial calculation function with specific calculations
            def initial_calculation(isin_data, temp_df, base_var, index):
                # Define the column name dynamically
                column_name = f'{base_var}_1.{index}'
                # Perform the calculation with the shift and weight in temp_df
                temp_df[column_name] = (
                    isin_data[f'{base_var}_NTM'].shift(index) * ((12 - index) / 12) +
                    isin_data[f'{base_var}_2yFwd'].shift(index) * (index / 12)
                )
            # Loop through each base_var to apply the calculation for both 'EPS' and 'Rev'
            for base_var in ['EPS', 'Rev']:
                calculate_momentum(isin_data, base_var=base_var, initial_calc_func=initial_calculation)
                
            ## currently this code is resulting in intermittend NaN values in the LS calc
            ## i think this is because of the nan error checker earlier in np.where line
            ## how do i want to treat this?
   
            # Start-of-month list
            SOM_columns = ['TR_LTM','TR_L6M','TR_L3M','NTM_RevGrowth','NTM_EBITDA_Margin','RoE_NTM','NTM_RevGrowth_3mChg',
                           'NTM_Rev_3mChg','NTM_EBITDA_Margin_3mChg','Sales_EV_NTM','EBITDA_EV_NTM','EY_NTM','EY_2yFwd','BY_NTM',
                           'CoE','Implied_G','Rev_MTUM_LTM','EPS_MTUM_LTM']
            
            for column in SOM_columns:
                isin_data[column + '_SOM'] = isin_data[column].shift(1)
            
            # Append the data to the list
            all_isin_data.append(isin_data)

# Concatenate all ISIN data into a single DataFrame
isin_final_df = pd.concat(all_isin_data, ignore_index=True)

# Merge the additional columns into the melted DataFrame using 'Date' and 'ISIN' as the keys 
df = pd.merge(df, isin_final_df, on=['Date','ISIN'])

df.tail()

#full_export_path = export_path + export_audit_file_name
#df.to_excel(full_export_path, index=False)


,Date,ISIN,Mkt_Cap,EV_NTM,RI,P,NOSH,Reccon,Rev_LTM,Rev_NTM,...,NTM_EBITDA_Margin_3mChg_SOM,Sales_EV_NTM_SOM,EBITDA_EV_NTM_SOM,EY_NTM_SOM,EY_2yFwd_SOM,BY_NTM_SOM,CoE_SOM,Implied_G_SOM,Rev_MTUM_LTM_SOM,EPS_MTUM_LTM_SOM
3387,2024-10-31,US7475251036,181325.90,190254.10,46468.25,162.77,1114.000737,2.24,38919.570,42507.465730,...,-0.004429,0.222417,0.084392,0.065451,0.071214,0.156601,0.095870,0.035916,0.011280,-0.015166
3388,2024-10-31,US83088M1027,13987.96,17509.50,27731.71,87.58,159.716374,2.80,4196.293,4257.032351,...,-0.015560,0.234227,0.075223,0.065101,0.079478,0.396578,0.100554,0.020993,-0.062322,-0.224335
3389,2024-10-31,US8740391003,988207.50,377883.30,6183.11,190.54,5186.351947,1.64,86075.440,108545.352200,...,0.008211,0.270663,0.185315,0.045152,0.054510,0.161167,0.083723,0.051627,0.157311,0.136863
3390,2024-10-31,US8807701029,17297.51,19399.39,9792.07,106.21,162.861407,2.36,2784.865,3257.768970,...,0.006052,0.149770,0.041760,0.031820,0.044437,0.143802,0.080927,0.055426,0.004469,-0.088316
3391,2024-10-31,US8825081040,185325.90,188827.70,24883.04,203.16,912.216480,2.82,15842.160,16922.930270,...,-0.000196,0.092802,0.044503,0.029385,0.035797,0.099937,0.075114,0.061829,0.017003,0.022074


In [2]:
#### CREATE DATAFRAME OF EQUAL WEIGHTED AND MARKET CAP WEIGHTED AVERAGE RATIOS ####


### Equal Weight Calculation
EqualWeightIndex_df = df.groupby('Date')[['TR',
                                          'NTM_RevGrowth','2y_RevCAGR',
                                          'NTM_EBITDA_Margin','2yFwd_EBITDA_Margin',
                                          'RoE_LTM', 'RoE_NTM', 'RoE_2yFwd',
                                          'CoE','Term_G','Implied_G',
                                          'Sales_EV_NTM','EBITDA_EV_NTM',
                                          'EY_LTM','EY_NTM','EY_2yFwd',
                                          'BY_LTM','BY_NTM','BY_2yFwd',
                                          'DY_LTM','DY_NTM','DY_2yFwd','NOSH_Chg_LTM',
                                          'Rev_MTUM_LTM','EPS_MTUM_LTM']].mean().reset_index()

EqualWeightIndex_df.columns = [f'{col}_Eq_Wgt' if col in ['TR',
                                          'NTM_RevGrowth','2y_RevCAGR',
                                          'NTM_EBITDA_Margin','2yFwd_EBITDA_Margin',
                                          'RoE_LTM', 'RoE_NTM', 'RoE_2yFwd',
                                          'CoE', 'Term_G','Implied_G',
                                          'Sales_EV_NTM','EBITDA_EV_NTM',
                                          'EY_LTM','EY_NTM','EY_2yFwd',
                                          'BY_LTM','BY_NTM','BY_2yFwd',
                                          'DY_LTM','DY_NTM','DY_2yFwd','NOSH_Chg_LTM',
                                          'Rev_MTUM_LTM','EPS_MTUM_LTM'] else col for col in EqualWeightIndex_df.columns]

EqualWeightIndex_df['EV_Sales_NTM_Eq_Wgt'] = 1/EqualWeightIndex_df['Sales_EV_NTM_Eq_Wgt']
EqualWeightIndex_df['EV_EBITDA_NTM_Eq_Wgt'] = 1/EqualWeightIndex_df['EBITDA_EV_NTM_Eq_Wgt']
EqualWeightIndex_df['PE_LTM_Eq_Wgt'] = 1/EqualWeightIndex_df['EY_LTM_Eq_Wgt']
EqualWeightIndex_df['PE_NTM_Eq_Wgt'] = 1/EqualWeightIndex_df['EY_NTM_Eq_Wgt']
EqualWeightIndex_df['PE_2yFwd_Eq_Wgt'] = 1/EqualWeightIndex_df['EY_2yFwd_Eq_Wgt']
EqualWeightIndex_df['PB_LTM_Eq_Wgt'] = 1/EqualWeightIndex_df['BY_LTM_Eq_Wgt']
EqualWeightIndex_df['PB_NTM_Eq_Wgt'] = 1/EqualWeightIndex_df['BY_NTM_Eq_Wgt']
EqualWeightIndex_df['PB_2yFwd_Eq_Wgt'] = 1/EqualWeightIndex_df['BY_2yFwd_Eq_Wgt']


### Median Weight Calculation
MedianIndex_df = df.groupby('Date')[['TR',
                                     'NTM_RevGrowth','2y_RevCAGR',
                                     'NTM_EBITDA_Margin','2yFwd_EBITDA_Margin',
                                     'RoE_LTM','RoE_NTM','RoE_2yFwd',
                                     'CoE','Term_G','Implied_G',
                                     'Sales_EV_NTM','EBITDA_EV_NTM',
                                     'EY_LTM','EY_NTM','EY_2yFwd',
                                     'BY_LTM','BY_NTM','BY_2yFwd',
                                     'DY_LTM','DY_NTM','DY_2yFwd','NOSH_Chg_LTM',
                                     'Rev_MTUM_LTM','EPS_MTUM_LTM']].median().reset_index()

MedianIndex_df.columns = [f'{col}_Median' if col in ['TR',
                                     'NTM_RevGrowth','2y_RevCAGR',
                                     'NTM_EBITDA_Margin','2yFwd_EBITDA_Margin',
                                     'RoE_LTM', 'RoE_NTM', 'RoE_2yFwd',
                                     'CoE', 'Term_G','Implied_G',
                                     'Sales_EV_NTM','EBITDA_EV_NTM',
                                     'EY_LTM','EY_NTM','EY_2yFwd',
                                     'BY_LTM','BY_NTM','BY_2yFwd',
                                     'DY_LTM','DY_NTM','DY_2yFwd','NOSH_Chg_LTM',
                                     'Rev_MTUM_LTM','EPS_MTUM_LTM'] else col for col in MedianIndex_df.columns]

MedianIndex_df['EV_Sales_NTM_Median'] = 1/MedianIndex_df['Sales_EV_NTM_Median']
MedianIndex_df['EV_EBITDA_NTM_Median'] = 1/MedianIndex_df['EBITDA_EV_NTM_Median']
MedianIndex_df['PE_LTM_Median'] = 1/MedianIndex_df['EY_LTM_Median']
MedianIndex_df['PE_NTM_Median'] = 1/MedianIndex_df['EY_NTM_Median']
MedianIndex_df['PE_2yFwd_Median'] = 1/MedianIndex_df['EY_2yFwd_Median']
MedianIndex_df['PB_LTM_Median'] = 1/MedianIndex_df['BY_LTM_Median']
MedianIndex_df['PB_NTM_Median'] = 1/MedianIndex_df['BY_NTM_Median']
MedianIndex_df['PB_2yFwd_Median'] = 1/MedianIndex_df['BY_2yFwd_Median']

### MKT CAP WEIGHTED CALCULATION

# Step 1: Create a function for weighted average calculation
def weighted_average(df, value_col, weight_col):
    valid_entries = ~df[value_col].isna()
    weights = df.loc[valid_entries, weight_col]
    values = df.loc[valid_entries, value_col]
    if weights.sum() == 0:
        # Handle the case where denominator is zero
        return 0
    else:
        weighted_avg = (weights * values).sum() / weights.sum()
        return weighted_avg

# Step 2: Create a function to calculate the market cap weighted averages
def calculate_weighted_averages(group):
    weighted_TR = weighted_average(group, 'TR', 'Mkt_Cap_SOM')
    weighted_NTM_RevGrowth = weighted_average(group, 'NTM_RevGrowth', 'Mkt_Cap')
    weighted_2y_RevCAGR = weighted_average(group, '2y_RevCAGR', 'Mkt_Cap')
    weighted_NTM_EBITDA_Margin = weighted_average(group, 'NTM_EBITDA_Margin', 'Mkt_Cap')
    weighted_2yFwd_EBITDA_Margin = weighted_average(group, '2yFwd_EBITDA_Margin', 'Mkt_Cap')
    weighted_RoE_LTM = weighted_average(group, 'RoE_LTM', 'Mkt_Cap')
    weighted_RoE_NTM = weighted_average(group, 'RoE_NTM', 'Mkt_Cap')
    weighted_RoE_2yFwd = weighted_average(group, 'RoE_2yFwd', 'Mkt_Cap')
    weighted_CoE = weighted_average(group, 'CoE', 'Mkt_Cap')
    weighted_Term_G = weighted_average(group, 'Term_G', 'Mkt_Cap')
    weighted_Implied_G = weighted_average(group, 'Implied_G', 'Mkt_Cap')
    weighted_Sales_EV_NTM = weighted_average(group, 'Sales_EV_NTM', 'Mkt_Cap')
    weighted_EBITDA_EV_NTM = weighted_average(group, 'EBITDA_EV_NTM', 'Mkt_Cap')
    weighted_EY_LTM = weighted_average(group, 'EY_LTM', 'Mkt_Cap')
    weighted_EY_NTM = weighted_average(group, 'EY_NTM', 'Mkt_Cap')
    weighted_EY_2yFwd = weighted_average(group, 'EY_2yFwd', 'Mkt_Cap') 
    weighted_BY_LTM = weighted_average(group, 'BY_LTM', 'Mkt_Cap')
    weighted_BY_NTM = weighted_average(group, 'BY_NTM', 'Mkt_Cap')
    weighted_BY_2yFwd = weighted_average(group, 'BY_2yFwd', 'Mkt_Cap') 
    weighted_DY_LTM = weighted_average(group, 'DY_LTM', 'Mkt_Cap')
    weighted_DY_NTM = weighted_average(group, 'DY_NTM', 'Mkt_Cap')
    weighted_DY_2yFwd = weighted_average(group, 'DY_2yFwd', 'Mkt_Cap')
    weighted_NOSH_Chg_LTM = weighted_average(group, 'NOSH_Chg_LTM', 'Mkt_Cap')
    weighted_Rev_MTUM_LTM = weighted_average(group, 'Rev_MTUM_LTM', 'Mkt_Cap')
    weighted_EPS_MTUM_LTM = weighted_average(group, 'EPS_MTUM_LTM', 'Mkt_Cap')
       
    return pd.Series({
        'TR_MV_Wgt': weighted_TR,
        'NTM_RevGrowth_MV_Wgt': weighted_NTM_RevGrowth,
        '2y_RevCAGR_MV_Wgt': weighted_2y_RevCAGR,
        'NTM_EBITDA_Margin_MV_Wgt': weighted_NTM_EBITDA_Margin,
        '2yFwd_EBITDA_Margin_MV_Wgt': weighted_2yFwd_EBITDA_Margin,
        'RoE_LTM_MV_Wgt': weighted_RoE_LTM,
        'RoE_NTM_MV_Wgt': weighted_RoE_NTM,
        'RoE_2yFwd_MV_Wgt': weighted_RoE_2yFwd,
        'CoE_MV_Wgt': weighted_CoE,
        'Term_G_MV_Wgt': weighted_Term_G,
        'Implied_G_MV_Wgt': weighted_Implied_G,
        'EV_Sales_NTM_MV_Wgt': weighted_Sales_EV_NTM, # harmonic mean; execution of reciprocal below
        'EV_EBITDA_NTM_MV_Wgt': weighted_EBITDA_EV_NTM, # harmonic mean; execution of reciprocal below
        'PE_LTM_MV_Wgt': weighted_EY_LTM, # harmonic mean; execution of reciprocal below
        'PE_NTM_MV_Wgt': weighted_EY_NTM, # harmonic mean; execution of reciprocal below
        'PE_2yFwd_MV_Wgt': weighted_EY_2yFwd, # harmonic mean; execution of reciprocal below
        'PB_LTM_MV_Wgt': weighted_BY_LTM, # harmonic mean; execution of reciprocal below
        'PB_NTM_MV_Wgt': weighted_BY_NTM, # harmonic mean; execution of reciprocal below
        'PB_2yFwd_MV_Wgt': weighted_BY_2yFwd, # harmonic mean; execution of reciprocal below
        'DY_LTM_MV_Wgt': weighted_DY_LTM,
        'DY_NTM_MV_Wgt': weighted_DY_NTM,
        'DY_2yFwd_MV_Wgt': weighted_DY_2yFwd,
        'NOSH_Chg_LTM_MV_Wgt': weighted_NOSH_Chg_LTM,
        'Rev_MTUM_LTM_MV_Wgt': weighted_Rev_MTUM_LTM,
        'EPS_MTUM_LTM_MV_Wgt': weighted_EPS_MTUM_LTM        
    })

# Step 3: Apply the calculation to your DataFrame
Index_df = df.groupby('Date').apply(calculate_weighted_averages).reset_index()

# Step 4: Convert multiples where necessary
Index_df['EV_Sales_NTM_MV_Wgt'] = 1/Index_df['EV_Sales_NTM_MV_Wgt']
Index_df['EV_EBITDA_NTM_MV_Wgt'] = 1/Index_df['EV_EBITDA_NTM_MV_Wgt']
Index_df['PE_LTM_MV_Wgt'] = 1/Index_df['PE_LTM_MV_Wgt']
Index_df['PE_NTM_MV_Wgt'] = 1/Index_df['PE_NTM_MV_Wgt']
Index_df['PE_2yFwd_MV_Wgt'] = 1/Index_df['PE_2yFwd_MV_Wgt']
Index_df['PB_LTM_MV_Wgt'] = 1/Index_df['PB_LTM_MV_Wgt']
Index_df['PB_NTM_MV_Wgt'] = 1/Index_df['PB_NTM_MV_Wgt']
Index_df['PB_2yFwd_MV_Wgt'] = 1/Index_df['PB_2yFwd_MV_Wgt']

# Step 4: Add in the equal weighted calculations from above into the Index dataframe
Index_df['TR_Eq_Wgt'] = EqualWeightIndex_df['TR_Eq_Wgt']
Index_df['NTM_RevGrowth_Eq_Wgt'] = EqualWeightIndex_df['NTM_RevGrowth_Eq_Wgt']
Index_df['2y_RevCAGR_Eq_Wgt'] = EqualWeightIndex_df['2y_RevCAGR_Eq_Wgt']
Index_df['NTM_EBITDA_Margin_Eq_Wgt'] = EqualWeightIndex_df['NTM_EBITDA_Margin_Eq_Wgt']
Index_df['2yFwd_EBITDA_Margin_Eq_Wgt'] = EqualWeightIndex_df['2yFwd_EBITDA_Margin_Eq_Wgt']
Index_df['RoE_LTM_Eq_Wgt'] = EqualWeightIndex_df['RoE_LTM_Eq_Wgt']
Index_df['RoE_NTM_Eq_Wgt'] = EqualWeightIndex_df['RoE_NTM_Eq_Wgt']
Index_df['RoE_2yFwd_Eq_Wgt'] = EqualWeightIndex_df['RoE_2yFwd_Eq_Wgt']
Index_df['CoE_Eq_Wgt'] = EqualWeightIndex_df['CoE_Eq_Wgt']
Index_df['Term_G_Eq_Wgt'] = EqualWeightIndex_df['Term_G_Eq_Wgt']
Index_df['Implied_G_Eq_Wgt'] = EqualWeightIndex_df['Implied_G_Eq_Wgt']
Index_df['EV_Sales_NTM_Eq_Wgt'] = EqualWeightIndex_df['EV_Sales_NTM_Eq_Wgt']
Index_df['EV_EBITDA_NTM_Eq_Wgt'] = EqualWeightIndex_df['EV_EBITDA_NTM_Eq_Wgt']
Index_df['PE_LTM_Eq_Wgt'] = EqualWeightIndex_df['PE_LTM_Eq_Wgt']
Index_df['PE_NTM_Eq_Wgt'] = EqualWeightIndex_df['PE_NTM_Eq_Wgt']
Index_df['PE_2yFwd_Eq_Wgt'] = EqualWeightIndex_df['PE_2yFwd_Eq_Wgt']
Index_df['PB_LTM_Eq_Wgt'] = EqualWeightIndex_df['PB_LTM_Eq_Wgt']
Index_df['PB_NTM_Eq_Wgt'] = EqualWeightIndex_df['PB_NTM_Eq_Wgt']
Index_df['PB_2yFwd_Eq_Wgt'] = EqualWeightIndex_df['PB_2yFwd_Eq_Wgt']
Index_df['DY_LTM_Eq_Wgt'] = EqualWeightIndex_df['DY_LTM_Eq_Wgt']
Index_df['DY_NTM_Eq_Wgt'] = EqualWeightIndex_df['DY_NTM_Eq_Wgt']
Index_df['DY_2yFwd_Eq_Wgt'] = EqualWeightIndex_df['DY_2yFwd_Eq_Wgt']
Index_df['NOSH_Chg_LTM_Eq_Wgt'] = EqualWeightIndex_df['NOSH_Chg_LTM_Eq_Wgt']
Index_df['Rev_MTUM_LTM_Eq_Wgt'] = EqualWeightIndex_df['Rev_MTUM_LTM_Eq_Wgt']
Index_df['EPS_MTUM_LTM_Eq_Wgt'] = EqualWeightIndex_df['EPS_MTUM_LTM_Eq_Wgt']

# Step 5: Add in the median calculations from above into the Index dataframe
Index_df['TR_Median'] = MedianIndex_df['TR_Median']
Index_df['NTM_RevGrowth_Median'] = MedianIndex_df['NTM_RevGrowth_Median']
Index_df['2y_RevCAGR_Median'] = MedianIndex_df['2y_RevCAGR_Median']
Index_df['NTM_EBITDA_Margin_Median'] = MedianIndex_df['NTM_EBITDA_Margin_Median']
Index_df['2yFwd_EBITDA_Margin_Median'] = MedianIndex_df['2yFwd_EBITDA_Margin_Median']
Index_df['RoE_LTM_Median'] = MedianIndex_df['RoE_LTM_Median']
Index_df['RoE_NTM_Median'] = MedianIndex_df['RoE_NTM_Median']
Index_df['RoE_2yFwd_Median'] = MedianIndex_df['RoE_2yFwd_Median']
Index_df['CoE_Median'] = MedianIndex_df['CoE_Median']
Index_df['Term_G_Median'] = MedianIndex_df['Term_G_Median']
Index_df['Implied_G_Median'] = MedianIndex_df['Implied_G_Median']
Index_df['EV_Sales_NTM_Median'] = MedianIndex_df['EV_Sales_NTM_Median']
Index_df['EV_EBITDA_NTM_Median'] = MedianIndex_df['EV_EBITDA_NTM_Median']
Index_df['PE_LTM_Median'] = MedianIndex_df['PE_LTM_Median']
Index_df['PE_NTM_Median'] = MedianIndex_df['PE_NTM_Median']
Index_df['PE_2yFwd_Median'] = MedianIndex_df['PE_2yFwd_Median']
Index_df['PB_LTM_Median'] = MedianIndex_df['PB_LTM_Median']
Index_df['PB_NTM_Median'] = MedianIndex_df['PB_NTM_Median']
Index_df['PB_2yFwd_Median'] = MedianIndex_df['PB_2yFwd_Median']
Index_df['DY_LTM_Median'] = MedianIndex_df['DY_LTM_Median']
Index_df['DY_NTM_Median'] = MedianIndex_df['DY_NTM_Median']
Index_df['DY_2yFwd_Median'] = MedianIndex_df['DY_2yFwd_Median']
Index_df['NOSH_Chg_LTM_Median'] = MedianIndex_df['NOSH_Chg_LTM_Median']
Index_df['Rev_MTUM_LTM_Median'] = MedianIndex_df['Rev_MTUM_LTM_Median']
Index_df['EPS_MTUM_LTM_Median'] = MedianIndex_df['EPS_MTUM_LTM_Median']


# Display the updated DataFrame
#Index_df.tail()

# Export to xlxs
full_export_path = export_path + export_index_file_name 
Index_df.to_excel(full_export_path, index=False)


In [3]:
### LONG / SHORT RETURNS BY VARIABLE ###

# List of target columns you want to iterate over
target_columns = ['Mkt_Cap_SOM','TR_LTM_SOM','TR_L6M_SOM','TR_L3M_SOM',
                  'NTM_RevGrowth_SOM','NTM_EBITDA_Margin_SOM','RoE_NTM_SOM','NTM_RevGrowth_3mChg_SOM','NTM_Rev_3mChg_SOM',
                  'NTM_EBITDA_Margin_3mChg_SOM','Sales_EV_NTM_SOM',
                  'EBITDA_EV_NTM_SOM','EY_NTM_SOM','EY_2yFwd_SOM','BY_NTM_SOM','CoE_SOM','Implied_G_SOM',
                  'Rev_MTUM_LTM_SOM','EPS_MTUM_LTM_SOM']            

# Initialize the final result DataFrame with unique Dates
LS_results_df = pd.DataFrame(df['Date'].unique(), columns=['Date'])

# Loop over each target column
for target_column in target_columns:
    
    # Ensure both the target column and 'TR' are numeric
    df[target_column] = pd.to_numeric(df[target_column], errors='coerce')
    df['TR'] = pd.to_numeric(df['TR'], errors='coerce')
    
    # List to collect results for this target column
    results = []
    
    # Group by 'Date' and compute median and means for 'Long_' and 'Short_' prefixed series
    for date, group in df.groupby('Date'):
        median_value = group[target_column].median()
        
        # Long: mean TR where target_column is above the median
        long_mean_tr = group[group[target_column] > median_value]['TR'].mean()
        
        # Short: mean TR where target_column is below the median
        short_mean_tr = group[group[target_column] < median_value]['TR'].mean()
        
        # Append the results for each date
        results.append({
            'Date': date,
            f'Long_{target_column}': long_mean_tr,  # Dynamically naming the column
            f'Short_{target_column}': short_mean_tr  # Dynamically naming the column
        })
    
    # Create a DataFrame with the results for the current target column
    target_df = pd.DataFrame(results)
    
    # Add the "LS_<target_column>" column
    target_df[f'LS_{target_column}'] = (1 + target_df[f'Long_{target_column}']) / (1 + target_df[f'Short_{target_column}']) - 1
    
    # Merge the current results with the final results DataFrame
    LS_results_df = pd.merge(LS_results_df, target_df, on='Date', how='left')

# Output the final result
#print(LS_results_df)

# Export to xlxs
full_export_path = export_path + export_LS_file_name 
LS_results_df.to_excel(full_export_path, index=False)

In [4]:
### VALUATION DISPERSION BY VARIABLE ###


# List of target columns you want to iterate over
target_columns = ['NTM_RevGrowth', '2y_RevCAGR',
                  'NTM_EBITDA_Margin','2yFwd_EBITDA_Margin',
                  'RoE_NTM','RoE_2yFwd',
                  'CoE','Term_G','Implied_G',
                  'Sales_EV_NTM','EBITDA_EV_NTM','EY_NTM','EY_2yFwd',
                  'Rev_MTUM_LTM','EPS_MTUM_LTM']

# Initialize the final result DataFrame with unique Dates
dispersion_results_df = pd.DataFrame(df['Date'].unique(), columns=['Date'])

# Loop over each target column
for target_column in target_columns:
    
    # Ensure both the target column and 'TR' are numeric
    df[target_column] = pd.to_numeric(df[target_column], errors='coerce')
    
    # List to collect results for this target column
    results = []
    
    # Group by 'Date' and compute median and means for 'Long_' and 'Short_' prefixed series
    for date, group in df.groupby('Date'):
        median_value = group[target_column].median()
        
        # Long: mean of target_column where target_column is above the median
        long_mean_target = group[group[target_column] > median_value][target_column].mean()
        
        # Short: mean of target_column where target_column is below the median
        short_mean_target = group[group[target_column] < median_value][target_column].mean()
        
        # Append the results for each date
        results.append({
            'Date': date,
            f'above_{target_column}': long_mean_target,  # Dynamically naming the column
            f'below_{target_column}': short_mean_target  # Dynamically naming the column
        })
    
    # Create a DataFrame with the results for the current target column
    target_df = pd.DataFrame(results)
    
    # Add the "LS_<target_column>" column
    target_df[f'spread_{target_column}'] = (1 + target_df[f'above_{target_column}']) / (1 + target_df[f'below_{target_column}']) - 1
    
    # Merge the current results with the final results DataFrame
    dispersion_results_df = pd.merge(dispersion_results_df, target_df, on='Date', how='left')

# Output the final result
#print(dispersion_results_df)

# Export to xlxs
full_export_path = export_path + export_dispersion_file_name 
dispersion_results_df.to_excel(full_export_path, index=False)

In [6]:
# examine stock by stock data for a single month
examine_data = df[(df['Date'] >= '2020-08-31') & (df['Date'] <= '2020-08-31')]

# Export to xlsx
full_export_path = export_path + export_examine_CSV_file_name
examine_data.to_excel(full_export_path, index=False)